In [1]:
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import growthepieapi_utils as gtp
import l2beat_utils as ltwo
sys.path.pop()

# # Usage
merged_df = gtp.get_growthepie_api_data()
merged_df = merged_df.reset_index()
print(merged_df.sample(5))

l2beat_df = ltwo.get_l2beat_activity_data()
print(l2beat_df.sample(5))

Failed to retrieve data for stables. HTTP Status code: 403
      index       date    chain_name       daa    txcount  txcosts_usd  \
2392   2392 2022-06-22  Arbitrum One   25927.0   180930.0     0.262577   
2667   2667 2023-03-24  Arbitrum One  420278.0  1814378.0     0.104386   
4234   4234 2023-07-05          Zora    1319.0     1920.0     0.496680   
4270   4270 2023-08-10          Zora   13100.0    29667.0     0.140603   
2724   2724 2023-05-20  Arbitrum One  159507.0   698109.0     0.165258   

      txcosts_eth       tvl_usd       tvl_eth       fees_usd    fees_eth  
2392     0.000233  2.123696e+09  1.888466e+06   64232.279001   57.117628  
2667     0.000057  6.754369e+09  3.701112e+06  262526.130564  143.853351  
4234     0.000256  9.196173e+05  4.747636e+02    1266.626350    0.653911  
4270     0.000076  3.558765e+06  1.917551e+03    8517.092644    4.589221  
2724     0.000091  6.418941e+09  3.542205e+06  155272.123117   85.684789  
       timestamp  transactions  ethereumTransa

In [ ]:
# Post to Dune API
d.write_dune_api_from_pandas(merged_df, 'growthepie_l2_activity',\
                             'L2 Usage Activity from GrowThePie')
d.write_dune_api_from_pandas(l2beat_df, 'l2beat_l2_activity',\
                             'L2 Usage Activity from L2Beat')